In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
from utils import *

In [ ]:
class VariantionalAutoencoder(object):
    def __init__(self, latent_dim, image_dim=28 * 28):
        self.hidden_encoder_dim = 256
        self.hidden_decoder_dim = 256
        self.latent_dim = latent_dim
        self.learning_rate = 1e-2

        self.image_dim = image_dim

        self.build_model()

    def build_model(self):
        self.x = tf.placeholder(name='x', dtype=tf.float32, shape=[None, self.image_dim])

        # Gaussian MLP as encoder
        with tf.variable_scope("gaussian_MLP_encoder"):
            he1 = tf.layers.dense(self.x, self.hidden_encoder_dim, activation=tf.nn.relu)
            he2 = tf.layers.dense(he1, self.hidden_encoder_dim, activation=tf.nn.relu)
            z_mu = tf.layers.dense(he2, self.latent_dim, activation=None)
            z_log_sigma = tf.layers.dense(he2, self.latent_dim, activation=None)
            self.z = ???

        # Bernolli MLP as decoder
        with tf.variable_scope("bernoulli_MLP_decoder"):
            hd1 = tf.layers.dense(self.z, self.hidden_decoder_dim, activation=tf.nn.relu)
            hd2 = tf.layers.dense(hd1, self.hidden_decoder_dim, activation=tf.nn.relu)
            self.x_hat = tf.layers.dense(hd2, self.image_dim, activation=tf.nn.sigmoid)

        with tf.variable_scope("loss"):
            # Reconstruction loss
            # Minimize the cross-entropy loss
            # H(x, x_hat) = -\Sigma x*log(x_hat) + (1-x)*log(1-x_hat)
            recon_loss = ???
            self.recon_loss = tf.reduce_mean(recon_loss)

            # Latent loss
            # Kullback Leibler divergence: measure the difference between two distributions
            # Here we measure the divergence between the latent distribution and N(0, 1)
            latent_loss = ???
            self.latent_loss = tf.reduce_mean(latent_loss)
            
            self.total_loss = tf.reduce_mean(recon_loss + latent_loss)
            self.train_op = tf.train.AdamOptimizer(
                learning_rate=self.learning_rate).minimize(self.total_loss)
            

In [ ]:
batch_size = 128
latent_dim = 100

f_mnist = input_data.read_data_sets('../data/fashion_mnist')

vae = VariantionalAutoencoder(latent_dim)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0
for it in range(100000):
    if it % 1000 == 0:
        samples = sess.run(vae.x_hat, feed_dict={vae.z: sample_z(16, latent_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    x_image, _ = f_mnist.train.next_batch(batch_size)

    _, loss, recon_loss, latent_loss = sess.run(
        [vae.train_op, vae.total_loss, vae.recon_loss, vae.latent_loss],
        feed_dict={vae.x: x_image}
    )

    if it % 1000 == 0:
        print('[Iter {}] Loss: {}, Recon loss: {}, Latent loss: {}'.format(
            it, loss, recon_loss, latent_loss))